<a href="https://colab.research.google.com/github/Avinash-ml07/Smart-Surveillance-Artist-Real-time-Security-Feed-with-Threat-Detection-Artistic-Stylization/blob/main/Security_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics
!pip install -q tensorflow opencv-python matplotlib


In [ ]:
from PIL import Image
import requests

url = "https://ultralytics.com/images/bus.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image.save("sample.jpg")
image.show()


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model('sample.jpg', save=True)

results[0].show()


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img('sample.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

model = ResNet50(weights='imagenet')
preds = model.predict(x)
print("Top Predictions:", decode_predictions(preds, top=3)[0])

print("\n🧱 Scene Classification by ResNet:")
for label in scene_preds:
    print(f"- {label[1]} ({round(label[2]*100, 2)}%)")


In [ ]:
print("Objects Detected:", detected_labels)
print("Scene Classification:", scene_preds[0][1])


In [ ]:
import requests
import os

content_path = "/content/input.jpg"
style_path = "/content/image.jpeg"

if os.path.exists(content_path) and os.path.exists(style_path):
    print("✅ Both images found")
else:
    print("❌ Missing image files")


In [ ]:
import tensorflow as tf


def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_jpeg(img, channels=3)  # 👈 Force JPEG decoding
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    scale = max_dim / max(shape)
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

content_image = load_img(content_path)
style_image = load_img(style_path)




In [ ]:
plt.subplot(1, 2, 1)
plt.title("Content")
plt.imshow(tf.squeeze(content_image))
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Style")
plt.imshow(tf.squeeze(style_image))
plt.axis('off')
plt.show()

In [ ]:
import tensorflow_hub as hub

hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

plt.figure(figsize=(6, 6))
plt.imshow(tf.squeeze(stylized_image))
plt.title("Stylized Output")
plt.axis('off')
plt.show()


In [ ]:
threat_objects = {"knife", "gun", "firearm", "weapon"}
found_threat = any(obj in detected_labels for obj in threat_objects)

if  found_threat:
    print("🚨 Threat detected! stylization.")
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
    plt.imshow(tf.squeeze(stylized_image))
    plt.title("Stylized Output")
    plt.axis('off')
    plt.show()
else:
    print("🚨 No Threat detected! Skipping stylization.")
    plt.imshow(tf.squeeze(content_image))
    plt.title("Stylized Output")
    plt.axis('off')
    plt.show()


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


In [ ]:
import torch
from IPython.display import display, Image
from PIL import Image as PILImage
import cv2
import numpy as np


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


In [ ]:
img = PILImage.open("input.jpg")

results = model(img)

results.print()
results.show()


In [ ]:
import torch
import cv2

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    cv2.imshow('YOLO Detection', np.squeeze(results.render()))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
abnormal_classes = ['knife', 'person', 'fire']

for *box, conf, cls in results.xyxy[0]:
    label = model.names[int(cls)]
    if label in abnormal_classes:
        print(f"⚠️ Abnormal object detected: {label}")
